In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dagshub
import mlflow
import mlflow.sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [4]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

In [5]:
dagshub.init(repo_owner='gioeba', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=9b0c9e4e-4083-4a45-ab11-06d43eee295e&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=2f8996988bce33aa968f9a54a105c07ee7cbe4e25826a2f01d93f59b0d028d05




Output()

Accessing as gioeba

Initialized MLflow to track repo "gioeba/IEEE-CIS-Fraud-Detection"

Repository gioeba/IEEE-CIS-Fraud-Detection initialized!

# Cleaning

In [6]:
df = train_transaction.merge(train_identity, on='TransactionID', how='left')
y = df['isFraud']
X = df.drop(columns=['isFraud', 'TransactionID']).copy()

# Feature Engineering

In [7]:
logged_fe = 'runs:/2251e67e9c084b99b4e492e146e576a0/fe'
feature_engineering_model = mlflow.sklearn.load_model(logged_fe)

# Feature Selection

In [8]:
logged_preprocessor = 'runs:/de686b00b14a4c7aac22431740c0ebb5/pre-processor'
dynamic_preprocessor = mlflow.sklearn.load_model(logged_preprocessor)

In [9]:
clf_pipeline = Pipeline([
    ('feature_engineering', feature_engineering_model),
    ('preprocessing', dynamic_preprocessor),
    ('classifier', xgb.XGBClassifier(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric='auc',
        use_label_encoder=False,
        tree_method='hist',
        scale_pos_weight=10,
        random_state=42,
        n_jobs=-1
    ))
])

# Training

In [10]:
mlflow.set_experiment("ieee-xgboost-experiment")
with mlflow.start_run(run_name="xgb-depth-8-n-100-prod"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        clf_pipeline.fit(X_train, y_train)
        y_prob = clf_pipeline.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_prob)
        auc_scores.append(auc)

        print(f"Fold {fold} AUC: {auc:.4f}")
        mlflow.log_metric(f"fold_{fold}_auc", auc)

    avg_auc = np.mean(auc_scores)
    print(f"\nAverage AUC across folds: {avg_auc:.4f}")

    mlflow.sklearn.log_model(clf_pipeline, artifact_path="xgboost-pipeline")
    
    
    mlflow.log_metric("avg_roc_auc", avg_auc)
    mlflow.log_params({
        "model": "xgboost",
        "n_estimators": 100,
        "max_depth": 8,
        "learning_rate": 0.05,
        "scale_pos_weight": 10
    })

Fold 1 AUC: 0.9268
Fold 2 AUC: 0.9274
Fold 3 AUC: 0.9247
Fold 4 AUC: 0.9261
Fold 5 AUC: 0.9252

Average AUC across folds: 0.9260


2025/04/27 08:02:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run xgb-depth-8-n-100-prod at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/8/runs/4eda508e9b104880bfea605c0044906a
🧪 View experiment at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/8
